In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_dict = load("/home/chuyu/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")

data_info_all = []
for key in keys(data_info_dict)
    append!(data_info_all, data_info_dict[key])
end

In [ ]:
analyzer = "chuyu"

In [ ]:
@showprogress for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    if candidate_filename[which_file] == []
        println(data_info)
    end

    file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
    all_files = readdir(file_folder_2)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    
    
    if candidate_filename[which_file] == []
        println(data_info)
    end

end

# original maps

In [ ]:
# first save the easily calculatable variables

@showprogress for which_data = 28:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)


    n_neuron = length(X_all);

    # whether individual roi belongs to a certain region
    region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
    region_bool_file = h5open(region_bool_filename, "r")
    global region_names = read(region_bool_file, "region_names")
    global region_roi_bool = read(region_bool_file, "region_roi_bool")
    close(region_bool_file)


    # for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
    region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
    whether_tel = falses(n_neuron)
    for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
        if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
            whether_tel[which_neuron] = true
        end
    end

    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)


    file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
    all_files = readdir(file_folder_2)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_2, save_file_name)
    file = h5open(info_filename, "r")
    specificity_2 = HDF5.readmmap(file["specificity"])
    specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_2 = HDF5.readmmap(file["valid_neurons"])
    place_map_all_2 = HDF5.readmmap(file["place_map_all"])
    close(file)

    n_bins = size(place_map_all_1, 1)




    place_cell_index_1 = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01))
    place_cell_index_2 = intersect(findall(specificity_population_z_2.>3), findall(specificity_shuffle_z_2.>5), findall(specificity_2.>0.01))

    tel_place_cell_index_1 = intersect(place_cell_index_1, findall(whether_tel))
    tel_place_cell_index_2 = intersect(place_cell_index_2, findall(whether_tel))

    place_cell_index = union(place_cell_index_1, place_cell_index_2)
    tel_place_cell_index = union(tel_place_cell_index_1, tel_place_cell_index_2) #what we should use in following analysis

    valid_index_1, peak_all_1 = neuron_with_valid_peak(tel_place_cell_index, place_map_all_1)
    valid_index_2, peak_all_2 = neuron_with_valid_peak(tel_place_cell_index, place_map_all_2)
    valid_index = intersect(valid_index_1, valid_index_2)

    confined_place_cell_index_1 = intersect(tel_place_cell_index[valid_index_1], tel_place_cell_index_1)
    confined_place_cell_index_2 = intersect(tel_place_cell_index[valid_index_2], tel_place_cell_index_2)
    confined_place_cell_index = intersect(confined_place_cell_index_1, confined_place_cell_index_2)

    peak_loc_map_1 = peak_all_1[:,whether_in(tel_place_cell_index, confined_place_cell_index_1)] 
    peak_loc_map_2 = peak_all_2[:,whether_in(tel_place_cell_index, confined_place_cell_index_2)];

    plot_loc(tel_place_cell_index_1; valid_neurons = valid_roi_1, label= length(tel_place_cell_index_1))
    plot_loc(tel_place_cell_index_2; valid_neurons = valid_roi_1, label= length(tel_place_cell_index_2))


    h5open(joinpath(file_folder_1, "compare_map_results_original.h5"), "w") do file
        file["specificity_1"] = specificity_1
        file["specificity_2"] = specificity_2
        file["tel_place_cell_index_1"] = tel_place_cell_index_1
        file["tel_place_cell_index_2"] = tel_place_cell_index_2
        file["tel_place_cell_index"] = tel_place_cell_index

        file["confined_place_cell_index_1"] = confined_place_cell_index_1
        file["confined_place_cell_index_2"] = confined_place_cell_index_2
        file["confined_place_cell_index"] = confined_place_cell_index
        file["place_cell_index_1"] = place_cell_index_1
        file["place_cell_index_2"] = place_cell_index_2

        file["peak_loc_map_1"] = peak_loc_map_1
        file["peak_loc_map_2"] = peak_loc_map_2
        file["peak_all_1"] = peak_all_1
        file["peak_all_2"] = peak_all_2
        
        file["whether_tel"] = collect(whether_tel)
        
    end;



end

In [ ]:
@showprogress for which_data = 1:length(data_info_all)
    
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    
    compare_map_file = h5open(joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5"))
    if haskey(compare_map_file, "pv_map_normalized")    
        continue
    end
    peak_loc_map_1 = read(compare_map_file,"peak_loc_map_1")
    peak_loc_map_2 = read(compare_map_file,"peak_loc_map_2")
    confined_place_cell_index_1 = read(compare_map_file,"confined_place_cell_index_1")
    confined_place_cell_index_2 = read(compare_map_file,"confined_place_cell_index_2")
    confined_place_cell_index = read(compare_map_file,"confined_place_cell_index")
    tel_place_cell_index = read(compare_map_file,"tel_place_cell_index")
    close(compare_map_file)
    
    
    

    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)


    file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
    all_files = readdir(file_folder_2)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_2, save_file_name)
    file = h5open(info_filename, "r")
    specificity_2 = HDF5.readmmap(file["specificity"])
    specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_2 = HDF5.readmmap(file["valid_neurons"])
    place_map_all_2 = HDF5.readmmap(file["place_map_all"])
    close(file)

    n_bins = size(place_map_all_1, 1)
    
    
    # orientation-corrected background image and chamber roi image
    orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
    global countour_array_1 = read(orientation_correction_file,"countour_array")
    close(orientation_correction_file)


    # orientation-corrected background image and chamber roi image
    orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_2), "chamber_geometry_$(experiment_filename_2).h5"))
    global countour_array_2 = read(orientation_correction_file,"countour_array")
    close(orientation_correction_file)


    img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
        read(file, "img_bg")
    end;

    img_bg_2 = h5open(ds_save_2, "behavior.h5"; raw = true) do file
        read(file, "img_bg")
    end;

    global img_bg_end_1 = img_bg_1[:,:,end]
    global img_bg_end_2 = img_bg_2[:,:,end]


    global w = size(img_bg_end_1, 1)
    global l = size(img_bg_end_1, 2)


    
    for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n$(n_bins).h5"))
    x_bins = read(for_place_calculation_file,"x_bins")
    y_bins = read(for_place_calculation_file,"y_bins")
    close(for_place_calculation_file)

    bin_interval = x_bins[2] - x_bins[1]

    KMeans_features = cluster.KMeans(n_clusters = 6).fit(hcat(peak_loc_map_1'))
    class_1 = KMeans_features.labels_;
    KMeans_features = cluster.KMeans(n_clusters = 6).fit(hcat(peak_loc_map_2'))
    class_2 = KMeans_features.labels_;

    fig = figure()
    scatter((peak_loc_map_1[1,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)].-0.5).*bin_interval, (peak_loc_map_1[2,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)].-0.5).*bin_interval, c=class_1[whether_in(confined_place_cell_index_1, confined_place_cell_index_1)], cmap="Dark2", s=10, alpha=0.3)
    # colorbar()
    imshow(img_bg_end_1', cmap="binary", vmax=2000)
    axis("off")
    title(experiment_filename_1)

    fig = figure()
    scatter((peak_loc_map_2[1,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)].-0.5).*bin_interval, (peak_loc_map_2[2,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)].-0.5).*bin_interval, c=class_1[whether_in(confined_place_cell_index_1, confined_place_cell_index_2)], cmap="Dark2", s=10, alpha=0.3)
    # colorbar()
    imshow(img_bg_end_2', cmap="binary", vmax=2000)
    axis("off")


    fig = figure()
    scatter((peak_loc_map_1[1,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)].-0.5).*bin_interval, (peak_loc_map_1[2,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)].-0.5).*bin_interval, c=class_2[whether_in(confined_place_cell_index_2, confined_place_cell_index_1)], cmap="Dark2", s=10, alpha=0.3)
    # colorbar()
    imshow(img_bg_end_1', cmap="binary", vmax=2000)
    axis("off")
    title(experiment_filename_1)

    fig = figure()
    scatter((peak_loc_map_2[1,whether_in(confined_place_cell_index_2, confined_place_cell_index_2)].-0.5).*bin_interval, (peak_loc_map_2[2,whether_in(confined_place_cell_index_2, confined_place_cell_index_2)].-0.5).*bin_interval, c=class_2[whether_in(confined_place_cell_index_2, confined_place_cell_index_2)], cmap="Dark2", s=10, alpha=0.3)
    # colorbar()
    imshow(img_bg_end_2', cmap="binary", vmax=2000)
    axis("off")

    # histogram of peak location change

    peak_shift = sqrt.(sum((peak_loc_map_1[:,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)] .- peak_loc_map_2[:,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)]) .^2, dims=1)[1,:])
    pixel_size = 20*10^-3
    peak_shift_mm = peak_shift *pixel_size*bin_interval

    # orientation-corrected background image and chamber roi image
    orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
    chamber_roi_1 = read(orientation_correction_file,"chamber_roi")
    center_loc_1 = read(orientation_correction_file,"center_loc")
    countour_array_1 = read(orientation_correction_file,"countour_array")
    close(orientation_correction_file)


    peak_loc_map_1_rough = (peak_loc_map_1.-0.5).*bin_interval
    peak_loc_map_2_rough = (peak_loc_map_2.-0.5).*bin_interval;


    peak_distance_center_1_mm = distance_from(peak_loc_map_1_rough', center_loc_1) .*pixel_size
    peak_distance_edge_1_mm,_ = distance_to_feature(collect(peak_loc_map_1_rough'), countour_array_1) .*pixel_size



    # histogram of PF correlation

    corr_trials_all = fill(NaN32, size(place_map_all_1,3))

    for neuron_idx in intersect(valid_roi_1, valid_roi_2)
        map_1 = place_map_all_1[:,:, neuron_idx]
        map_2 = place_map_all_2[:,:, neuron_idx]
        corr_trials_all[neuron_idx] = corr_2d(map_1, map_2)
    end


    # histogram of PV correlation
    which_neuron = tel_place_cell_index
    mask_valid_1 = isfinite.(place_map_all_1[:,:,confined_place_cell_index[1]])
    mask_valid_2 = isfinite.(place_map_all_2[:,:,confined_place_cell_index[1]])
    valid_pixels = findall(mask_valid_1 .* mask_valid_2)

    mean_map_1= nanmean(place_map_all_1[valid_pixels, which_neuron], dims=1)[1,:]
    std_map_1= nanstd(place_map_all_1[valid_pixels, which_neuron], dims=1)[1,:]
    mean_map_2= nanmean(place_map_all_2[valid_pixels, which_neuron], dims=1)[1,:]
    std_map_2= nanstd(place_map_all_2[valid_pixels, which_neuron], dims=1)[1,:]


    corr_pv_all = fill(NaN32, length(valid_pixels))
    corr_pv_all_normalized = fill(NaN32, length(valid_pixels))
    for (i, which_pixel) in enumerate(valid_pixels)
        pv1 = place_map_all_1[which_pixel, which_neuron]
        pv2 = place_map_all_2[which_pixel, which_neuron]
        pv1_normalized = (pv1 .- mean_map_1)./std_map_1
        pv2_normalized = (pv2 .- mean_map_2)./std_map_2
        corr_pv_all[i] = corr_nan(pv1, pv2)
        corr_pv_all_normalized[i] = corr_nan(pv1_normalized, pv2_normalized)
    end


    pv_map = fill(NaN32, size(place_map_all_1[:,:,1]))
    pv_map[valid_pixels] = corr_pv_all


    pv_map_normalized = fill(NaN32, size(place_map_all_1[:,:,1]))
    pv_map_normalized[valid_pixels] = corr_pv_all_normalized

    h5open(joinpath(file_folder_1, "compare_map_results_original.h5"), "r+") do file
        file["peak_shift_mm"] = peak_shift_mm
        file["peak_distance_center_1_mm"] = peak_distance_center_1_mm
        file["peak_distance_edge_1_mm"] = peak_distance_edge_1_mm
        file["corr_trials_all"] = corr_trials_all
        file["corr_pv_all"] = corr_pv_all
        file["corr_pv_all_normalized"] = corr_pv_all_normalized
        file["pv_map"] = pv_map
        file["pv_map_normalized"] = pv_map_normalized
    end;



end

# Correct for wrong normalization

In [ ]:
@showprogress for which_data = 1:length(data_info_all)
    
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    
    compare_map_file = h5open(joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5"))

    peak_loc_map_1 = read(compare_map_file,"peak_loc_map_1")
    peak_loc_map_2 = read(compare_map_file,"peak_loc_map_2")
    confined_place_cell_index_1 = read(compare_map_file,"confined_place_cell_index_1")
    confined_place_cell_index_2 = read(compare_map_file,"confined_place_cell_index_2")
    confined_place_cell_index = read(compare_map_file,"confined_place_cell_index")
    tel_place_cell_index = read(compare_map_file,"tel_place_cell_index")
    close(compare_map_file)
    
    
    

    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)


    file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
    all_files = readdir(file_folder_2)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_2, save_file_name)
    file = h5open(info_filename, "r")
    specificity_2 = HDF5.readmmap(file["specificity"])
    specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_2 = HDF5.readmmap(file["valid_neurons"])
    place_map_all_2 = HDF5.readmmap(file["place_map_all"])
    close(file)

    n_bins = size(place_map_all_1, 1)
    
    
    # orientation-corrected background image and chamber roi image
    orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
    global countour_array_1 = read(orientation_correction_file,"countour_array")
    close(orientation_correction_file)


    # orientation-corrected background image and chamber roi image
    orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_2), "chamber_geometry_$(experiment_filename_2).h5"))
    global countour_array_2 = read(orientation_correction_file,"countour_array")
    close(orientation_correction_file)


    img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
        read(file, "img_bg")
    end;

    img_bg_2 = h5open(ds_save_2, "behavior.h5"; raw = true) do file
        read(file, "img_bg")
    end;

    global img_bg_end_1 = img_bg_1[:,:,end]
    global img_bg_end_2 = img_bg_2[:,:,end]


    global w = size(img_bg_end_1, 1)
    global l = size(img_bg_end_1, 2)


    
    for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n$(n_bins).h5"))
    x_bins = read(for_place_calculation_file,"x_bins")
    y_bins = read(for_place_calculation_file,"y_bins")
    close(for_place_calculation_file)

    bin_interval = x_bins[2] - x_bins[1]

    KMeans_features = cluster.KMeans(n_clusters = 6).fit(hcat(peak_loc_map_1'))
    class_1 = KMeans_features.labels_;
    KMeans_features = cluster.KMeans(n_clusters = 6).fit(hcat(peak_loc_map_2'))
    class_2 = KMeans_features.labels_;

    fig = figure()
    scatter((peak_loc_map_1[1,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)].-0.5).*bin_interval, (peak_loc_map_1[2,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)].-0.5).*bin_interval, c=class_1[whether_in(confined_place_cell_index_1, confined_place_cell_index_1)], cmap="Dark2", s=10, alpha=0.3)
    # colorbar()
    imshow(img_bg_end_1', cmap="binary", vmax=2000)
    axis("off")
    title(experiment_filename_1)

    fig = figure()
    scatter((peak_loc_map_2[1,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)].-0.5).*bin_interval, (peak_loc_map_2[2,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)].-0.5).*bin_interval, c=class_1[whether_in(confined_place_cell_index_1, confined_place_cell_index_2)], cmap="Dark2", s=10, alpha=0.3)
    # colorbar()
    imshow(img_bg_end_2', cmap="binary", vmax=2000)
    axis("off")


    fig = figure()
    scatter((peak_loc_map_1[1,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)].-0.5).*bin_interval, (peak_loc_map_1[2,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)].-0.5).*bin_interval, c=class_2[whether_in(confined_place_cell_index_2, confined_place_cell_index_1)], cmap="Dark2", s=10, alpha=0.3)
    # colorbar()
    imshow(img_bg_end_1', cmap="binary", vmax=2000)
    axis("off")
    title(experiment_filename_1)

    fig = figure()
    scatter((peak_loc_map_2[1,whether_in(confined_place_cell_index_2, confined_place_cell_index_2)].-0.5).*bin_interval, (peak_loc_map_2[2,whether_in(confined_place_cell_index_2, confined_place_cell_index_2)].-0.5).*bin_interval, c=class_2[whether_in(confined_place_cell_index_2, confined_place_cell_index_2)], cmap="Dark2", s=10, alpha=0.3)
    # colorbar()
    imshow(img_bg_end_2', cmap="binary", vmax=2000)
    axis("off")

    # histogram of peak location change

    peak_shift = sqrt.(sum((peak_loc_map_1[:,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)] .- peak_loc_map_2[:,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)]) .^2, dims=1)[1,:])
    pixel_size = 20*10^-3
    peak_shift_mm = peak_shift *pixel_size*bin_interval

    # orientation-corrected background image and chamber roi image
    orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
    chamber_roi_1 = read(orientation_correction_file,"chamber_roi")
    center_loc_1 = read(orientation_correction_file,"center_loc")
    countour_array_1 = read(orientation_correction_file,"countour_array")
    close(orientation_correction_file)


    peak_loc_map_1_rough = (peak_loc_map_1.-0.5).*bin_interval
    peak_loc_map_2_rough = (peak_loc_map_2.-0.5).*bin_interval;


    peak_distance_center_1_mm = distance_from(peak_loc_map_1_rough', center_loc_1) .*pixel_size
    peak_distance_edge_1_mm,_ = distance_to_feature(collect(peak_loc_map_1_rough'), countour_array_1) .*pixel_size



    # histogram of PF correlation

    corr_trials_all = fill(NaN32, size(place_map_all_1,3))

    for neuron_idx in intersect(valid_roi_1, valid_roi_2)
        map_1 = place_map_all_1[:,:, neuron_idx]
        map_2 = place_map_all_2[:,:, neuron_idx]
        corr_trials_all[neuron_idx] = corr_2d_original(map_1, map_2)
    end


    # histogram of PV correlation
    which_neuron = tel_place_cell_index
    mask_valid_1 = isfinite.(place_map_all_1[:,:,confined_place_cell_index[1]])
    mask_valid_2 = isfinite.(place_map_all_2[:,:,confined_place_cell_index[1]])
    valid_pixels = findall(mask_valid_1 .* mask_valid_2)
        
    mean_map_1= numpy.nanmean(place_map_all_1[mask_valid_1, which_neuron], axis=0)
    std_map_1= numpy.nanstd(place_map_all_1[mask_valid_1, which_neuron], axis=0)
    mean_map_2= numpy.nanmean(place_map_all_2[mask_valid_2, which_neuron], axis=0)
    std_map_2= numpy.nanstd(place_map_all_2[mask_valid_2, which_neuron], axis=0)

    corr_pv_all = fill(NaN32, length(valid_pixels))
    corr_pv_all_normalized = fill(NaN32, length(valid_pixels))
    for (i, which_pixel) in enumerate(valid_pixels)
        pv1 = place_map_all_1[which_pixel, which_neuron]
        pv2 = place_map_all_2[which_pixel, which_neuron]
        pv1_normalized = (pv1 .- mean_map_1)./std_map_1
        pv2_normalized = (pv2 .- mean_map_2)./std_map_2
        corr_pv_all[i] = corr_nan(pv1, pv2)
        corr_pv_all_normalized[i] = corr_nan(pv1_normalized, pv2_normalized)
    end


    pv_map = fill(NaN32, size(place_map_all_1[:,:,1]))
    pv_map[valid_pixels] = corr_pv_all


    pv_map_normalized = fill(NaN32, size(place_map_all_1[:,:,1]))
    pv_map_normalized[valid_pixels] = corr_pv_all_normalized
    
    figure()
    imshow(pv_map_normalized)

    h5open(joinpath(file_folder_1, "compare_map_results_original.h5"), "r+") do file
        
        delete_object(file, "corr_trials_all")
        delete_object(file, "corr_pv_all")
        delete_object(file, "corr_pv_all_normalized")
        delete_object(file, "pv_map")
        delete_object(file, "pv_map_normalized")
        file["corr_trials_all"] = corr_trials_all
        file["corr_pv_all"] = corr_pv_all
        file["corr_pv_all_normalized"] = corr_pv_all_normalized
        file["pv_map"] = pv_map
        file["pv_map_normalized"] = pv_map_normalized
    end;



end

# Correct for PV correlation

In [ ]:
@showprogress for which_data = 1:length(data_info_all)
    
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    
    compare_map_file = h5open(joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5"))

    peak_loc_map_1 = read(compare_map_file,"peak_loc_map_1")
    peak_loc_map_2 = read(compare_map_file,"peak_loc_map_2")
    confined_place_cell_index_1 = read(compare_map_file,"confined_place_cell_index_1")
    confined_place_cell_index_2 = read(compare_map_file,"confined_place_cell_index_2")
    confined_place_cell_index = read(compare_map_file,"confined_place_cell_index")
    tel_place_cell_index = read(compare_map_file,"tel_place_cell_index")
    close(compare_map_file)
    
    
    

    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)


    file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
    all_files = readdir(file_folder_2)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_2, save_file_name)
    file = h5open(info_filename, "r")
    specificity_2 = HDF5.readmmap(file["specificity"])
    specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_2 = HDF5.readmmap(file["valid_neurons"])
    place_map_all_2 = HDF5.readmmap(file["place_map_all"])
    close(file)



    n_bins = size(place_map_all_1, 1)
    n_x = size(place_map_all_1, 1)
    n_y = size(place_map_all_1, 2) 
    n_neurons = size(place_map_all_1, 3)
    map_baseline_1 = fill(NaN32, n_neurons)
    map_baseline_2 = fill(NaN32, n_neurons);

    for neuron_idx = tel_place_cell_index
        cur_map = place_map_all_1[:, :, neuron_idx]
        map_baseline_1[neuron_idx] = nanmean(cur_map[findall(cur_map .<= nanpctile(cur_map, 20))])
        cur_map = place_map_all_2[:, :, neuron_idx]
        map_baseline_2[neuron_idx] = nanmean(cur_map[findall(cur_map .<= nanpctile(cur_map, 20))])
    end

    pseudocount_1 = 10 .- map_baseline_1;
    pseudocount_1[findall(pseudocount_1 .< 0)] .= 0; 
    pseudocount_2 = 10 .- map_baseline_2;
    pseudocount_2[findall(pseudocount_2 .< 0)] .= 0; 


    place_activity_map_dFoF_1 = fill(NaN32, n_x, n_y, n_neurons)
    place_activity_map_dFoF_2 = fill(NaN32, n_x, n_y, n_neurons)

    PV_correlation_map = fill(NaN32, n_x, n_y)

    for neuron_idx = tel_place_cell_index
        cur_map = place_map_all_1[:, :, neuron_idx]
        cur_map_dFoF = (cur_map .- map_baseline_1[neuron_idx]) ./ (map_baseline_1[neuron_idx] + pseudocount_1[neuron_idx])
        place_activity_map_dFoF_1[:, :, neuron_idx] .= cur_map_dFoF

        cur_map = place_map_all_2[:, :, neuron_idx]
        cur_map_dFoF = (cur_map .- map_baseline_2[neuron_idx]) ./ (map_baseline_2[neuron_idx] + pseudocount_2[neuron_idx])
        place_activity_map_dFoF_2[:, :, neuron_idx] .= cur_map_dFoF
    end

    for i_x = 1:n_x
        for i_y = 1:n_y
            PV_correlation_map[i_x, i_y] = nancor(place_activity_map_dFoF_1[i_x, i_y, tel_place_cell_index], place_activity_map_dFoF_2[i_x, i_y, tel_place_cell_index])
        end
    end


    h5open(joinpath(file_folder_1, "compare_map_results_original.h5"), "r+") do file
        file["pv_map_dFF"] = PV_correlation_map
    end;
end

# Correct for PF correlation

In [ ]:
@showprogress for which_data = 1:length(data_info_all)
    
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    
    compare_map_file = h5open(joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5"))

    peak_loc_map_1 = read(compare_map_file,"peak_loc_map_1")
    peak_loc_map_2 = read(compare_map_file,"peak_loc_map_2")
    confined_place_cell_index_1 = read(compare_map_file,"confined_place_cell_index_1")
    confined_place_cell_index_2 = read(compare_map_file,"confined_place_cell_index_2")
    confined_place_cell_index = read(compare_map_file,"confined_place_cell_index")
    tel_place_cell_index = read(compare_map_file,"tel_place_cell_index")
    close(compare_map_file)
    
    
    

    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)


    file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
    all_files = readdir(file_folder_2)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_2, save_file_name)
    file = h5open(info_filename, "r")
    specificity_2 = HDF5.readmmap(file["specificity"])
    specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_2 = HDF5.readmmap(file["valid_neurons"])
    place_map_all_2 = HDF5.readmmap(file["place_map_all"])
    close(file)

    n_bins = size(place_map_all_1, 1)
    


    corr_trials_all = fill(NaN32, size(place_map_all_1,3))

    for neuron_idx in intersect(valid_roi_1, valid_roi_2)
        map_1 = place_map_all_1[:,:, neuron_idx]
        map_2 = place_map_all_2[:,:, neuron_idx]
        corr_trials_all[neuron_idx] = corr_2d_original(map_1, map_2)
    end



    h5open(joinpath(file_folder_1, "compare_map_results_original.h5"), "r+") do file
        
        delete_object(file, "corr_trials_all")
        file["corr_trials_all"] = corr_trials_all

    end;



end

In [ ]:
@showprogress for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    compare_map_file = h5open(joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5"))
    if haskey(compare_map_file, "pv_map_normalized")    
        println("$which_data yes")
    else 
        println("$which_data no")
    end
    close(compare_map_file)

end

In [ ]:
@showprogress for which_data = 1:length(data_info_all)
    
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    
    compare_map_file = h5open(joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5"))

    place_cell_index_1 = read(compare_map_file,"place_cell_index_1")
    place_cell_index_2 = read(compare_map_file,"place_cell_index_2")
    close(compare_map_file)
    
    
    println(length(place_cell_index_1))
    println(length(place_cell_index_2))



end